In [45]:
import pandas as pd

# Load the dataset
file_path = '../raw_data/raw_election_results.csv'  # Update with your file path
df = pd.read_csv(file_path)

# Filter the dataset for UK-national results for the specified election years (2005, 2010, 2015, 2017, 2019)
filtered_df = df[(df['Country'] == 'United Kingdom') & (df['Year'].isin([2005, 2010, 2015, 2017, 2019]))]

# Drop the specified columns except 'Seats'
columns_to_drop = ['Vote Change', 'Percent Change', 'Seat Change', 'Percent Seat Change']
filtered_df_dropped = filtered_df.drop(columns=columns_to_drop)

# Pivot the table for votes
pivot_votes_df = filtered_df_dropped.pivot_table(
    index=['Year', 'Geography', 'Country', 'Region or Nation', 'County'],
    columns='Party',
    values='Votes',
    fill_value=0
).reset_index()

# Pivot the table for seats
pivot_seats_df = filtered_df_dropped.pivot_table(
    index=['Year', 'Geography', 'Country', 'Region or Nation', 'County'],
    columns='Party',
    values='Seats',
    fill_value=0
).reset_index()

# Flatten the column hierarchy created by pivoting
pivot_votes_df.columns.name = None
pivot_seats_df.columns.name = None

# Perform a join on the pivoted votes and seats dataframes
pivot_df = pivot_votes_df.join(pivot_seats_df.set_index(['Year', 'Geography', 'Country', 'Region or Nation', 'County']),
                               on=['Year', 'Geography', 'Country', 'Region or Nation', 'County'],
                               lsuffix='_Votes', rsuffix='_Seats')

# Add a column that tallies the total votes for each year
pivot_df['Total Votes'] = pivot_df.filter(like='_Votes').sum(axis=1)

# Calculate the percentage of total votes for each party
parties = [
    'The Brexit Party', 'Conservative', 'Green Party', 'Liberal Democrats',
    'Labour', 'Plaid Cymru', 'SNP', 'UK Independence Party (UKIP)']

for party in parties:
    pivot_df[f'{party} Percentage'] = (pivot_df[f'{party}_Votes'] / pivot_df['Total Votes']) * 100

# Select only the relevant columns for the specified parties and the associated data
selected_parties = [
    'The Brexit Party', 'Conservative', 'Green Party', 'Liberal Democrats',
    'Labour', 'Plaid Cymru', 'SNP', 'UK Independence Party (UKIP)', 'Total Votes'
]

# Columns to select: votes, seats, and percentage columns for the specified parties
selected_columns = ['Year', 'Geography', 'Country']  # Base columns
for party in selected_parties:
    if party == 'Total Votes':
        selected_columns.append(party)
    else:
        selected_columns.append(f'{party}_Votes')
        selected_columns.append(f'{party}_Seats')
        selected_columns.append(f'{party} Percentage')

# Filter the dataframe to include only the selected columns
filtered_pivot_df = pivot_df[selected_columns]
#Renames DF
df = filtered_pivot_df
# Define a dictionary to rename the columns
rename_dict = {
    'Conservative_Votes': 'CON_ACTUAL_VOTES',
    'Conservative_Seats': 'CON_ACTUAL_SEATS',
    'Conservative Percentage': 'CON_ACTUAL_PERCENTAGE',
    'Labour_Votes': 'LABOUR_ACTUAL_VOTES',
    'Labour_Seats': 'LABOUR_ACTUAL_SEATS',
    'Labour Percentage': 'LABOUR_ACTUAL_PERCENTAGE',
    'Liberal Democrats_Votes': 'LIB_ACTUAL_VOTES',
    'Liberal Democrats_Seats': 'LIB_ACTUAL_SEATS',
    'Liberal Democrats Percentage': 'LIB_ACTUAL_PERCENTAGE',
    'The Brexit Party_Votes': 'BRX_ACTUAL_VOTES',
    'The Brexit Party_Seats': 'BRX_ACTUAL_SEATS',
    'The Brexit Party Percentage': 'BRX_ACTUAL_PERCENTAGE',
    'Green Party_Votes': 'GRE_ACTUAL_VOTES',
    'Green Party_Seats': 'GRE_ACTUAL_SEATS',
    'Green Party Percentage': 'GRE_ACTUAL_PERCENTAGE',
    'Plaid Cymru_Votes': 'PLC_ACTUAL_VOTES',
    'Plaid Cymru_Seats': 'PLC_ACTUAL_SEATS',
    'Plaid Cymru Percentage': 'PLC_ACTUAL_PERCENTAGE',
    'SNP_Votes': 'SNP_ACTUAL_VOTES',
    'SNP_Seats': 'SNP_ACTUAL_SEATS',
    'SNP Percentage': 'SNP_ACTUAL_PERCENTAGE',
    'UK Independence Party (UKIP)_Votes': 'UKI_ACTUAL_VOTES',
    'UK Independence Party (UKIP)_Seats': 'UKI_ACTUAL_SEATS',
    'UK Independence Party (UKIP) Percentage': 'UKI_ACTUAL_PERCENTAGE'
}

In [46]:
# Renames columns, then dataframe
filtered_pivot_df.rename(columns=rename_dict, inplace=True)
df = filtered_pivot_df

/tmp/ipykernel_84283/1736961953.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_pivot_df.rename(columns=rename_dict, inplace=True)


In [47]:
df

,Year,Geography,Country,BRX_ACTUAL_VOTES,BRX_ACTUAL_SEATS,BRX_ACTUAL_PERCENTAGE,CON_ACTUAL_VOTES,CON_ACTUAL_SEATS,CON_ACTUAL_PERCENTAGE,GRE_ACTUAL_VOTES,...,PLC_ACTUAL_VOTES,PLC_ACTUAL_SEATS,PLC_ACTUAL_PERCENTAGE,SNP_ACTUAL_VOTES,SNP_ACTUAL_SEATS,SNP_ACTUAL_PERCENTAGE,UKI_ACTUAL_VOTES,UKI_ACTUAL_SEATS,UKI_ACTUAL_PERCENTAGE,Total Votes
0,2005,UK,United Kingdom,0.0,0.0,0.000000,8784816.0,198.0,32.359595,257654.0,...,174838.0,3.0,0.644030,412267.0,6.0,1.518620,605973.0,0.0,2.232152,27147484.0
1,2010,UK,United Kingdom,0.0,0.0,0.000000,10703764.0,306.0,36.053929,265243.0,...,165394.0,3.0,0.557103,491386.0,6.0,1.655156,919915.0,0.0,3.098588,29688204.0
2,2015,UK,United Kingdom,0.0,0.0,0.000000,11299592.0,330.0,36.810989,1111524.0,...,181704.0,3.0,0.591942,1454436.0,56.0,4.738156,3881081.0,1.0,12.643503,30696247.0
3,2017,UK,United Kingdom,0.0,0.0,0.000000,13635786.0,317.0,42.342718,512327.0,...,164466.0,4.0,0.510710,977568.0,35.0,3.035607,594067.0,0.0,1.844735,32203379.0
4,2019,UK,United Kingdom,644257.0,0.0,2.012681,13966454.0,365.0,43.631688,836108.0,...,153265.0,4.0,0.478805,1242380.0,48.0,3.881238,22817.0,0.0,0.071281,32009887.0


In [48]:
# Calculates other
df['OTH_ACTUAL_VOTES'] = df ['Total Votes'] - df['BRX_ACTUAL_VOTES'] - df['CON_ACTUAL_VOTES'] - df['GRE_ACTUAL_VOTES'] - df['LIB_ACTUAL_VOTES'] - df['LABOUR_ACTUAL_VOTES'] - df['PLC_ACTUAL_VOTES'] - df['SNP_ACTUAL_VOTES'] - df['UKI_ACTUAL_VOTES']
df['OTH_SEATs'] = 650 - df['BRX_ACTUAL_SEATS']  - df['CON_ACTUAL_SEATS'] - df['GRE_ACTUAL_SEATS'] - df['LIB_ACTUAL_SEATS'] - df['LABOUR_ACTUAL_SEATS'] - df['PLC_ACTUAL_SEATS'] - df['SNP_ACTUAL_SEATS'] - df['UKI_ACTUAL_SEATS']
df['OTH_PERCENTAGE'] = 100 - df['BRX_ACTUAL_PERCENTAGE'] - df['CON_ACTUAL_PERCENTAGE'] - df['GRE_ACTUAL_PERCENTAGE'] - df['LIB_ACTUAL_PERCENTAGE'] - df['LABOUR_ACTUAL_PERCENTAGE'] - df['PLC_ACTUAL_PERCENTAGE'] - df['SNP_ACTUAL_PERCENTAGE'] - df['UKI_ACTUAL_PERCENTAGE']

/tmp/ipykernel_84283/736825141.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['OTH_ACTUAL_VOTES'] = df ['Total Votes'] - df['BRX_ACTUAL_VOTES'] - df['CON_ACTUAL_VOTES'] - df['GRE_ACTUAL_VOTES'] - df['LIB_ACTUAL_VOTES'] - df['LABOUR_ACTUAL_VOTES'] - df['PLC_ACTUAL_VOTES'] - df['SNP_ACTUAL_VOTES'] - df['UKI_ACTUAL_VOTES']
/tmp/ipykernel_84283/736825141.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['OTH_SEATs'] = 650 - df['BRX_ACTUAL_SEATS']  - df['CON_ACTUAL_SEATS'] - df['GRE_ACTUAL_SEATS'] -

In [49]:
# Renames dataframe
election_results_df = df

In [50]:
election_results_df

,Year,Geography,Country,BRX_ACTUAL_VOTES,BRX_ACTUAL_SEATS,BRX_ACTUAL_PERCENTAGE,CON_ACTUAL_VOTES,CON_ACTUAL_SEATS,CON_ACTUAL_PERCENTAGE,GRE_ACTUAL_VOTES,...,SNP_ACTUAL_VOTES,SNP_ACTUAL_SEATS,SNP_ACTUAL_PERCENTAGE,UKI_ACTUAL_VOTES,UKI_ACTUAL_SEATS,UKI_ACTUAL_PERCENTAGE,Total Votes,OTH_ACTUAL_VOTES,OTH_SEATs,OTH_PERCENTAGE
0,2005,UK,United Kingdom,0.0,0.0,0.000000,8784816.0,198.0,32.359595,257654.0,...,412267.0,6.0,1.518620,605973.0,0.0,2.232152,27147484.0,1380116.0,26.0,5.083771
1,2010,UK,United Kingdom,0.0,0.0,0.000000,10703764.0,306.0,36.053929,265243.0,...,491386.0,6.0,1.655156,919915.0,0.0,3.098588,29688204.0,1699145.0,19.0,5.723300
2,2015,UK,United Kingdom,0.0,0.0,0.000000,11299592.0,330.0,36.810989,1111524.0,...,1454436.0,56.0,4.738156,3881081.0,1.0,12.643503,30696247.0,1004766.0,19.0,3.273254
3,2017,UK,United Kingdom,0.0,0.0,0.000000,13635786.0,317.0,42.342718,512327.0,...,977568.0,35.0,3.035607,594067.0,0.0,1.844735,32203379.0,1069341.0,19.0,3.320586
4,2019,UK,United Kingdom,644257.0,0.0,2.012681,13966454.0,365.0,43.631688,836108.0,...,1242380.0,48.0,3.881238,22817.0,0.0,0.071281,32009887.0,1179136.0,19.0,3.683662


In [51]:
# Trims dataframe down to just percentage column so that it can be used in the next step
percentage_columns = ['Year', 'Geography', 'Country']
percentage_columns += [col for col in filtered_pivot_df.columns if 'PERCENTAGE' in col]

# Filter the dataframe to include only the selected columns
percentage_df = election_results_df[percentage_columns]

In [52]:
# Exports to CSV - option 1
percentage_df.to_csv('../processed_data/2004_to_2019_results_clean.csv', index=False)

In [ ]:
# Exports to CSV - option 2
# percentage_df.to_csv('../processed_data/2019_to_2024_results_clean.csv', index=False)